# User Study 02 - RL Audio Notebook

<span style="color:red">**Before starting this Notebook...**</span>

1. <span style="color:red">**In Settings:**</span> Enable JupyterLab Dark. Go to Settings --> Theme --> "JupyterLab Dark" (*Optional but recommended*)

2. <span style="color:red">**In Setup Cells:**</span> Substitute the line of code specifying PWD path on your device to: ./../RL_audio/notebooks

3. <span style="color:red">**In Setup Cells:**</span> Install the required packages to run the notebook



# Setup Cells ~ These are run by the study administrator to setup the notebook

In [1]:
# %cd /home/liamroy/Documents/PHD/repos/RL_audio/notebooks
%cd '/Users/liamroy/liam.roy@monash.edu - Google Drive/My Drive/Robotics/User Studies/study_02/RL_audio/notebooks'
# %cd <add your path here and comment out the others>

/Users/liamroy/Library/CloudStorage/GoogleDrive-liam.roy@monash.edu/My Drive/Robotics/User Studies/study_02/RL_audio/notebooks


In [2]:
PWD = %pwd

## Packages

Please install the following:

pygame   (see this webpage ~ https://www.pygame.org/wiki/GettingStarted)  
jupyterlab, numpy, termcolor, openpyxl, nbconvert-webpdf  


Either use:    
--> sudo apt-get install <package_name>  
--> python3 -m pip install <package_name>  
--> conda install -c conda-forge <package_name>  


Example using conda:  
--> conda install -c conda-forge <package_name>  

jupyterlab or notebook  
numpy  
termcolor  
openpyxl  
nbconvert-webpdf              

## Imports

In [3]:
import os
import shutil
import time
import numpy as np
import random
import argparse
import linecache

from scripts import audio_control_V2 as audio_ctrl
from scripts import ucb1_algorithm_V2 as ucb1
from scripts import misc_helpers_V2 as mischelp

import sys

from termcolor import colored, cprint
# Termcolor guide: https://pypi.org/project/termcolor/

## Initializations

In [4]:
# Parameter discritization
param_disc = 3 

state_descriptions = ["Stuck	  \t- robot is in trouble and needs your help",         #Index [0]
                      "Accomplished \t- robot has successfully completed it's task",    #Index [1]
                      "Progressing \t- robot is working and doesn't need help",         #Index [2]
                      "None of the above"]                                              #Index [3]


num_of_states = len(state_descriptions) - 1 # Adding a minus 1 since the last state in "state_descriptions" is "none of the above"
state_range = np.arange(num_of_states)


# CREATE SOUND LIBRARY A
# For library A, setup the array using libA
library_A = "libA"

# Create an array of size (N x N x N) where N = number of discretized regions
# number of discretized regions for each param --> i.e. if equals 3 then (0, 1, 2)
# ** must align with the discretization for selected sound library
sound_obj_array_A = np.ndarray((param_disc, param_disc, param_disc),dtype=object)

for param_1_range in range(param_disc):
	for param_2_range in range(param_disc):
		for param_3_range in range(param_disc):
			sound_obj_array_A[param_1_range, param_2_range, param_3_range] = audio_ctrl.audio_object(param_1=param_1_range, param_2=param_2_range, param_3=param_3_range, sound_library=library_A)
			
			
# CREATE SOUND LIBRARY B
# For library B, setup the array using libB
library_B = "libB"

# Create an array of size (N x N x N) where N = number of discretized regions
# number of discretized regions for each param --> i.e. if equals 3 then (0, 1, 2)
# ** must align with the discretization for selected sound library
sound_obj_array_B = np.ndarray((param_disc, param_disc, param_disc),dtype=object)

for param_1_range in range(param_disc):
	for param_2_range in range(param_disc):
		for param_3_range in range(param_disc):
			sound_obj_array_B[param_1_range, param_2_range, param_3_range] = audio_ctrl.audio_object(param_1=param_1_range, param_2=param_2_range, param_3=param_3_range, sound_library=library_B)
			

# ** <span style="color:red">Note To Reviewers</span> **

We have updated this notebook with comments explaining the study workflow, cell by cell. These cells have been added to describe the study procedure to enable reproducibility. 

The comments we provide are added in the cells with the header **<span style="color:red">COMMENTS FOR REVIEWERS</span>**

An example is shown in the cell below:

____________________________________________________________________

**<span style="color:red">COMMENTS FOR REVIEWERS</span>**

This is an example cell which has been added to aid reviewers in understanding the study procedure. 

This cell was **NOT** in the original Notebook.

____________________________________________________________________

# Forms & Pre-Survey <span style="color:yellow">(users start here)</span>

Please click the following two links to read the explanatory statement and answer the pre-study questionnaire.

<span style="color:yellow">**Explanatory Statement:**</span> https://drive.google.com/file/d/1-8npbW1wg_ABzBnnGa1dgEgCaYjDED8o/view?usp=sharing

<span style="color:yellow">**Consent Form:**</span> https://forms.gle/eZMYzwaHG4Bqpgp6A

<span style="color:yellow">**Pre-study Questionnaire:**</span> https://forms.gle/GAU8xzekWKkTMDLVA   (Participant ID Required)

____________________________________________________________________

**<span style="color:red">COMMENTS FOR REVIEWERS</span>**

Here we had participants enter their participant ID. This ID was provided to them by the study administrator supervising them as they completed the notebook.
____________________________________________________________________

Then enter your user ID:
	
<span style="color:yellow">**Click on the first cell below & hit 'shift + enter'...**</span>

In [5]:



current_user_ID_str = mischelp.get_user_ID(parent_dir=PWD, num_of_states=num_of_states)





Please enter your double digiet user ID number (ex: 00 ) then hit 'enter'
 00





------------------------------------------------------------------------
------------------------------------------------------------------------

Great job! You are user: 00

Click on the next cell below and hit 'shift + enter' to continue

------------------------------------------------------------------------
------------------------------------------------------------------------





____________________________________________________________________

**<span style="color:red">COMMENTS FOR REVIEWERS</span>**

After participants had read the explanatory statement, signed the consent form, and completed the pre-survey, they returned to the notebook and started working through the cells below.

Throughout the entire survey, participants sat in a room with a study administrator. The administrator supervised the completion of the survey, ensuring that participants followed the notebook without modifying any of the notebook cells. The administrator also answered any basic questions posed by the participants regarding how to use the notebook.
____________________________________________________________________

# MAIN STUDY <span style="color:yellow">(start here after you've filled the pre-survey)</span>

Welcome to this study's <span style="color:yellow">**Jupyter notebook**</span>. In this work, we are developing strategies for improving human-robot interaction with nonverbal sounds (<span style="color:yellow">**_beeps & boops_**</span>).

This study is best completed with <span style="color:yellow">**headphones**</span>. Ensure your volume is on.

While a robot is working on a task, it can have many different internal states. Consider a scenario in which you are guiding a robot as it navigates a maze to find fruit:  

If the robot gets lost or stuck behind an obstacle, the robot's internal state is: <span style="color:Red">**Stuck**</span>

Similarly, if the robot was able to reach it's goal (find a fruit), the robot's internal state is: <span style="color:green">**Accomplished**</span>

If the robot is actively working on the task (navigating through the maze) but has neither gotten stuck nor completed the task, the robot's internal state is: <span style="color:blue">**Progressing**</span>

In this notebook, you will be asked to run through <span style="color:yellow">**3 tasks**</span>. In each of these tasks, a virtual robot will play a sound. Once you listen to the sound, you will be asked to select which robot state you think the virtual robot is in. You will have the options: <span style="color:Red">**Stuck**</span>, <span style="color:green">**Accomplished**</span>, <span style="color:blue">**Progressing**</span> and <span style="color:orange">**None of the above**</span>.

In addition to each answer, you will also self-score how confident you are in your response, on a scale from 1 to 10. 

This process will repeat several times as a learning algorithm is processing in the background. <span style="color:yellow">**If you have any questions, ask your study moderator**</span>. Have fun!

# Task 1

____________________________________________________________________

**<span style="color:red">COMMENTS FOR REVIEWERS</span>**

Task 1 serves as a pre-algorithm assessment to collect users' accuracy in correctly inferring states before running the algorithm.
____________________________________________________________________

## Subtask JBL

____________________________________________________________________

**<span style="color:red">COMMENTS FOR REVIEWERS</span>**

In subtask JBL, users are presented with an image of the Jackal robot (shown below). The Jackal robot expresses three different sounds signifying three different functional states (Stuck, Accomplished, Progressing). The order in which these sounds are presented to the participants is randomized.

The sounds used in this subtask are those from library A: https://github.com/liamreneroy/RL_audio/tree/main/notebooks/audio/sonif_libA

The parameter values used for each sound are shown here: https://github.com/liamreneroy/RL_audio/tree/main/notebooks/arrays

After listening to each sound, participants are asked to specify which state they think that sound represents. The options they have to choose from are Stuck, Accomplished, Progressing and None of the above.

In every scenario where participants go through this process of listening to sounds and specifying states, they are allowed to listen to the sound for as long as they want and replay it as many times as they would like. 
____________________________________________________________________

<span style="color:yellow">**Our first robot is named Jackal.**</span>

Let's listen to <span style="color:yellow">**Jackal**</span> make a few sounds to express itself. 

For each sound, you will asked to select which robot state you think the robot is in.

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

In [6]:



mischelp.get_user_accuracy(sound_obj_array=sound_obj_array_A, lib_str=library_A, sect_str="sect1", user_ID_str=current_user_ID_str, num_of_states=num_of_states, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file="pilotset", seed=70,  mixer_volume=0.75)




------------------------------------------------------------------------
------------------------------------------------------------------------

Robot sound is playing....

What state is the robot in: 

[S]: Stuck	  	- robot is in trouble and needs your help 
[A]: Accomplished 	- robot has successfully completed it's task 
[P]: Progressing 	- robot is working and doesn't need help 
[N]: None of the above 


To replay the sound: leave the input empty and hit 'enter'...
Select a state by entering its first letter [S - A - P - N]: 


 P



You entered: P
Progressing 	- robot is working and doesn't need help

To replay the sound: Leave the input empty and hit 'enter'...
Score your confidence in this response from [0 to 10]
 or 
Type 'back' to change your response: 


 8



You entered: 8

------------------------------------------------------------------------
------------------------------------------------------------------------

Robot sound is playing....

What state is the robot in: 

[S]: Stuck	  	- robot is in trouble and needs your help 
[A]: Accomplished 	- robot has successfully completed it's task 
[P]: Progressing 	- robot is working and doesn't need help 
[N]: None of the above 


To replay the sound: leave the input empty and hit 'enter'...
Select a state by entering its first letter [S - A - P - N]: 


...


You entered: 9

------------------------------------------------------------------------
------------------------------------------------------------------------

Robot sound is playing....

What state is the robot in: 

[S]: Stuck	  	- robot is in trouble and needs your help 
[A]: Accomplished 	- robot has successfully completed it's task 
[P]: Progressing 	- robot is working and doesn't need help 
[N]: None of the above 


To replay the sound: leave the input empty and hit 'enter'...
Select a state by entering its first letter [S - A - P - N]: 


 S



You entered: S
Stuck	  	- robot is in trouble and needs your help

To replay the sound: Leave the input empty and hit 'enter'...
Score your confidence in this response from [0 to 10]
 or 
Type 'back' to change your response: 


 6



You entered: 6




------------------------------------------------------------------------
------------------------------------------------------------------------

Great job!
Click on the next cell below and hit 'shift + enter' to continue

------------------------------------------------------------------------
------------------------------------------------------------------------





<img src="images/jackal.png" alt="Jackal Robot" style="height: 334px; width:600px;"/>

## Subtask SBL

____________________________________________________________________

**<span style="color:red">COMMENTS FOR REVIEWERS</span>**

In subtask SBL, users are presented with an image of the Spot robot (shown below). Similar to JBL, The Spot robot expresses three different sounds signifying three different functional states (Stuck, Accomplished, Progressing). The order in which these sounds are presented to the participants is randomized.

The sounds used in this subtask are those from library B: https://github.com/liamreneroy/RL_audio/tree/main/notebooks/audio/sonif_libB

The parameter values used for each sound are the same as JBL, shown here: https://github.com/liamreneroy/RL_audio/tree/main/notebooks/arrays

Similar to JBL, after listening to each sound, participants are asked to specify which state they think that sound represents. The options they have to choose from are Stuck, Accomplished, Progressing and None of the above.
____________________________________________________________________

<span style="color:yellow">**Our next robot is named the Spot.**</span>

Let's listen to <span style="color:yellow">**Spot**</span> make a few sounds to express itself. 

You will notice <span style="color:yellow">**Spot**</span> sounds slightly different to <span style="color:yellow">**Jackal**</span>. For each sound, you will asked to select which robot state you think the robot is in.

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

In [8]:


mischelp.get_user_accuracy(sound_obj_array=sound_obj_array_B, lib_str=library_B, sect_str="sect1", user_ID_str=current_user_ID_str, num_of_states=num_of_states, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file="pilotset", seed=51,  mixer_volume=0.45)




------------------------------------------------------------------------
------------------------------------------------------------------------

Robot sound is playing....

What state is the robot in: 

[S]: Stuck	  	- robot is in trouble and needs your help 
[A]: Accomplished 	- robot has successfully completed it's task 
[P]: Progressing 	- robot is working and doesn't need help 
[N]: None of the above 


To replay the sound: leave the input empty and hit 'enter'...
Select a state by entering its first letter [S - A - P - N]: 


 A



You entered: A
Accomplished 	- robot has successfully completed it's task

To replay the sound: Leave the input empty and hit 'enter'...
Score your confidence in this response from [0 to 10]
 or 
Type 'back' to change your response: 


 6



You entered: 6

------------------------------------------------------------------------
------------------------------------------------------------------------

Robot sound is playing....

What state is the robot in: 

[S]: Stuck	  	- robot is in trouble and needs your help 
[A]: Accomplished 	- robot has successfully completed it's task 
[P]: Progressing 	- robot is working and doesn't need help 
[N]: None of the above 


To replay the sound: leave the input empty and hit 'enter'...
Select a state by entering its first letter [S - A - P - N]: 


...


You entered: 7

------------------------------------------------------------------------
------------------------------------------------------------------------

Robot sound is playing....

What state is the robot in: 

[S]: Stuck	  	- robot is in trouble and needs your help 
[A]: Accomplished 	- robot has successfully completed it's task 
[P]: Progressing 	- robot is working and doesn't need help 
[N]: None of the above 


To replay the sound: leave the input empty and hit 'enter'...
Select a state by entering its first letter [S - A - P - N]: 


 S



You entered: S
Stuck	  	- robot is in trouble and needs your help

To replay the sound: Leave the input empty and hit 'enter'...
Score your confidence in this response from [0 to 10]
 or 
Type 'back' to change your response: 


 4



You entered: 4




------------------------------------------------------------------------
------------------------------------------------------------------------

Great job!
Click on the next cell below and hit 'shift + enter' to continue

------------------------------------------------------------------------
------------------------------------------------------------------------





<img src="images/spot.png" alt="Spot Robot" style="height: 334px; width:600px;"/>

# Task 2

In Task 2, we'll be listening to <span style="color:yellow">**Jackal**</span> again.

Similar to before, <span style="color:yellow">**Jackal**</span> make a few sounds to express itself, and you will asked to select which robot state you think the robot is in.

This process will repeat several times as a learning algorithm is processing in the background.

____________________________________________________________________

**<span style="color:red">COMMENTS FOR REVIEWERS</span>**

We are now in the second task of the study, where we implement the learning algorithm. In this task, users complete two subtasks (Subtask U: Uninformed Initialization and Subtask I: Informed Initialization). 

Participants were randomly assigned to complete Subtask Uninformed Initialization (U) or Informed Initialization (I) first. This randomization aimed to mitigate potential ordering bias. Participants who performed Subtasks U then Subtask I followed Condition UI, while those who completed Subtasks I then Subtask U followed Condition IU. 

In both subtasks, the Jackal (sound library A) used the proposed learning algorithm to learn suitable parameter values for three functional states. These Subtasks U/I function by presenting a sound to the participant and then asking them to specify which state they think that sound represents. The options they have to choose from are Stuck, Accomplished, Progressing and None of the above. Their response is used to formulate a reward signal, which is then used to update all of the Q-tables (recall, there is a Q-table for each robot state).

To streamline Q-table updates, we propagate the reward signal from a single action exploration to update corresponding Q-values across all robot states in the learning process. When a user associates a sound with the parameter combination $[1, 2, 0]$ to denote the "Stuck" state, the corresponding Q-value in the "Stuck" Q-table is updated positively, whereas Q-tables for all other states receive negative updates.

The algorithm in Subtasks U/I iteratively updates each state's Q-table until one of two conditions is met: the system reaches convergence or uses up its budget B. In this study, the budget was B=60, therefor the subtask could continue this interactive process of presenting a state and asking for a response up to 60 times. The details of this algorithm, how convergence is defined, and the hyper-parameters used are detailed in our research paper. 

As stated above, participants are allowed to listen to the sound for as long as they want and replay it as many times as they would like.

Subtasks U and I differed in how the Q-tables for each robot state were initialized, as described below.

Note that users are unaware of what 'Subtask U' or 'Subtask I' represents. 
____________________________________________________________________

## Subtask I

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

____________________________________________________________________

**<span style="color:red">COMMENTS FOR REVIEWERS</span>**

In Subtask I: Informed Initialization, the Jackal robot (sound library A) used the proposed learning algorithm to learn suitable parameter values for three functional states. Since the robot was learning three functional states (Stuck, Accomplished, and Progressing), three Q-tables were initialized (one for each state).

Under informed initialization, the Q-values are set to predefined values derived from prior knowledge. These predefined values range from -10 to 10 and were derived from user-suggested sound representations for each robot state in prior research. In informed initialization, a green cube (positive value) signifies users' previous identification of a sound as appropriate for the target state, whereas an orange cube (negative value) indicates a sound was deemed less appropriate.

The parameter values used as the initialization point for each state are shown here: https://github.com/liamreneroy/RL_audio/tree/main/notebooks/arrays
____________________________________________________________________

In [ ]:
# Subtask I


ucb1.ucb1_algor(num_of_states=num_of_states, state_descriptions=state_descriptions, param_disc=param_disc, 
                sound_obj_array=sound_obj_array_A, current_user_ID_str=current_user_ID_str, sect_str="_sect2O", load_file="pilotset", budget=60, 
                delta_Q_thresh=2.0, conv_thresh=3, printer=None, mixer_volume=0.75)


sect3_load_str = current_user_ID_str + "_sect2O_final"

<img src="images/jackal.png" alt="Jackal Robot" style="height: 334px; width:600px;"/>

## Subtask U

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

____________________________________________________________________

**<span style="color:red">COMMENTS FOR REVIEWERS</span>**

In Subtask U: Uninformed Initialization, the Jackal robot (sound library A) used the proposed learning algorithm to learn suitable parameter values for three functional states. Since the robot was learning three functional states (Stuck, Accomplished, and Progressing), three Q-tables were initialized (one for each state).

Following an uninformed initialization, all values within the Q-table are initialized to their maximum possible value (in this case $10$) following the principle of optimism under uncertainty. With all Q-values initialized to their maximum, the algorithm's argmax function forces the exploration of all actions before returning to those yielding the highest rewards.

The parameter values used as the initialization point for each state are shown here: https://github.com/liamreneroy/RL_audio/tree/main/notebooks/arrays
____________________________________________________________________

In [ ]:
# Subtask U


ucb1.ucb1_algor(num_of_states=num_of_states, state_descriptions=state_descriptions, param_disc=param_disc, 
                sound_obj_array=sound_obj_array_A, current_user_ID_str=current_user_ID_str, sect_str="_sect2X", load_file=None, budget=60, 
                delta_Q_thresh=2.0, conv_thresh=3, printer=None, mixer_volume=0.75)


sect3_load_str = current_user_ID_str + "_sect2X_final"

<img src="images/jackal.png" alt="Jackal Robot" style="height: 334px; width:600px;"/>

# Task 3

____________________________________________________________________

**<span style="color:red">COMMENTS FOR REVIEWERS</span>**

Task 3 serves as a post-algorithm assessment to compare users' accuracy in correctly inferring states after running the algorithm.

Users are again presented with images of two robots: Jackal (Subtask JAL) and Spot (Subtask SAL). In both subtasks, the robot communicated its three functional states (Stuck, Accomplished, and Progressing) using parameters learned from the algorithm in Task 2. If a participant completed condition UI, the parameters learned in Subtask I were used for Task 3. If they completed condition IU, then the parameters learned in Subtask U were used for Task 3.
____________________________________________________________________

## Subtask JAL

____________________________________________________________________

**<span style="color:red">COMMENTS FOR REVIEWERS</span>**

Similar subtask JBL, the sounds used in this subtask are those from library A:

https://github.com/liamreneroy/RL_audio/tree/main/notebooks/audio/sonif_libA

After listening to each sound, participants are asked to specify which state they think that sound represents. The options they have to choose from are Stuck, Accomplished, Progressing and None of the above.

Participants are allowed to listen to the sound for as long as they want and replay it as many times as they would like. 
____________________________________________________________________

We're nearly finished ~ <span style="color:yellow">**home stretch!**</span>

Let's listen to <span style="color:yellow">**Jackal**</span> express itself one last time. 

For each sound, you will asked to select which robot state you think the robot is in.

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

In [ ]:


mischelp.get_user_accuracy(sound_obj_array=sound_obj_array_A, lib_str=library_A, sect_str="sect3", user_ID_str=current_user_ID_str, num_of_states=num_of_states, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file=sect3_load_str, seed=51, mixer_volume=0.75)




<img src="images/jackal.png" alt="Jackal Robot" style="height: 334px; width:600px;"/>

## Subtask SAL

____________________________________________________________________

**<span style="color:red">COMMENTS FOR REVIEWERS</span>**

Similar subtask SBL, the sounds used in this subtask are those from library B:

https://github.com/liamreneroy/RL_audio/tree/main/notebooks/audio/sonif_libB

After listening to each sound, participants are asked to specify which state they think that sound represents. The options they have to choose from are Stuck, Accomplished, Progressing and None of the above.

Participants are allowed to listen to the sound for as long as they want and replay it as many times as they would like. 
____________________________________________________________________

Lastly, let's listen to <span style="color:yellow">**Spot**</span> express itself one last time.  

You will notice <span style="color:yellow">**Spot**</span> sounds slightly different to <span style="color:yellow">**Jackal**</span>. For each sound, you will asked to select which robot state you think the robot is in.

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

In [ ]:



mischelp.get_user_accuracy(sound_obj_array=sound_obj_array_B, lib_str=library_B, sect_str="sect3", user_ID_str=current_user_ID_str, num_of_states=num_of_states, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file=sect3_load_str, seed=48, mixer_volume=0.45)




<img src="images/spot.png" alt="Spot Robot" style="height: 334px; width:600px;"/>

## Final Results: <span style="color:yellow">**Jackal**</span>

Lets listen to the sounds the Jackal learned to express each of it's 3 states...

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

____________________________________________________________________

**<span style="color:red">COMMENTS FOR REVIEWERS</span>**

In the final results cells, we play the Jackal (Library A) and Spot (Library B) sounds based on the final parameters that were learned. This is simply done to allow participants to get a sense of how well the robot learned. This helps them respond to the post-survey :)! 
____________________________________________________________________

In [ ]:
# State 0 - Stuck


mischelp.play_final_sound(sound_obj_array=sound_obj_array_A, lib_str=library_A, sect_str="sect3", user_ID_str=current_user_ID_str, given_state_idx=0, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file=sect3_load_str, seed=48, mixer_volume=0.45)




In [ ]:
# State 1 - Accomplished


mischelp.play_final_sound(sound_obj_array=sound_obj_array_A, lib_str=library_A, sect_str="sect3", user_ID_str=current_user_ID_str, given_state_idx=1, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file=sect3_load_str, seed=48, mixer_volume=0.45)




In [ ]:
# State 2 - Progressing


mischelp.play_final_sound(sound_obj_array=sound_obj_array_A, lib_str=library_A, sect_str="sect3", user_ID_str=current_user_ID_str, given_state_idx=2, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file=sect3_load_str, seed=48, mixer_volume=0.45)




## Final Results: <span style="color:yellow">**Spot**</span>

Lets listen to the sounds the Jackal learned to express each of it's 3 states...

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

In [ ]:
# State 0 - Stuck


mischelp.play_final_sound(sound_obj_array=sound_obj_array_B, lib_str=library_B, sect_str="sect3", user_ID_str=current_user_ID_str, given_state_idx=0, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file=sect3_load_str, seed=48, mixer_volume=0.45)




In [ ]:
# State 1 - Accomplished


mischelp.play_final_sound(sound_obj_array=sound_obj_array_B, lib_str=library_B, sect_str="sect3", user_ID_str=current_user_ID_str, given_state_idx=1, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file=sect3_load_str, seed=48, mixer_volume=0.45)




In [ ]:
# State 2 - Progressing


mischelp.play_final_sound(sound_obj_array=sound_obj_array_B, lib_str=library_B, sect_str="sect3", user_ID_str=current_user_ID_str, given_state_idx=2, 
                           states_array=np.ndarray(num_of_states, dtype=object), state_descriptions=state_descriptions, param_disc=param_disc, load_file=sect3_load_str, seed=48, mixer_volume=0.45)




# Save the Output 

Run the following code block to save the output of this Jupyter Notebook.

<span style="color:yellow">**Click on the cell below & hit 'shift + enter'...**</span>

In [ ]:

file_path_name = "user_data/user_" + current_user_ID_str + "/final_output"

cmd = "jupyter nbconvert --to webpdf --allow-chromium-download study_notebook_V4.ipynb --output " + file_path_name
if(os.system(cmd)):
	print("Error converting to .py")
	print(f"cmd: {cmd}")
    

# Closing Survey

Please click the folliwng link to answer a short post-study questionnaire.

<span style="color:yellow">**Pre-study Questionnaire:**</span> https://forms.gle/K6RnncY82vSVdyE38   (Participant ID Required)

Thank you for completing this Jupyter Notebook. 

### NOTES & DEBUG

<span style="color:red">**These cells are not part of the survey.**</span>

In [ ]:
# PILOTSET ARRAY VALUE SETTER

# State 0: Stuck - Pilot Set
manual_Qtable_state_0 = np.array([[[1., -1., -3.], [2., 0., -3.], [3., 2., -3.]], 
								  [[2., -1., -3.], [2., 0., -3.], [4., 2., -3.]],
								  [[2., -1., -3.], [3., 0., -3.], [5., 3., -3.]]]) * 1.0

print("State 0: Stuck")
print(manual_Qtable_state_0.shape, "\n")
print(manual_Qtable_state_0, "\n")



# State 1: Successful - Pilot Set
manual_Qtable_state_1 = np.array([[[-3., 0., 2.], [-3., 1., 3.], [-3., 0., 2.]], 
								  [[-3., 0., 4.], [-3., 1., 5.], [-3., 0., 3.]],
								  [[-3., 0., 2.], [-3., 1., 3.], [-3., 0., 2.]]]) * 1.0

print("State 1: Successful")
print(manual_Qtable_state_1.shape, "\n")
print(manual_Qtable_state_1, "\n")



# State 2: Progressing - Pilot Set
manual_Qtable_state_2 = np.array([[[0., 3., 0.], [-3., 2., -3.], [-3., 1., -3.]], 
								  [[0., 5., 0.], [-3., 3., -3.], [-3., 1., -3.]],
								  [[0., 4., 0.], [-3., 2., -3.], [-3., 1., -3.]]]) * 1.0

print("State 2: Successful")
print(manual_Qtable_state_2.shape, "\n")
print(manual_Qtable_state_2, "\n")



np.save("arrays/pilotset_st0.npy", manual_Qtable_state_0)
np.save("arrays/pilotset_st1.npy", manual_Qtable_state_1)
np.save("arrays/pilotset_st2.npy", manual_Qtable_state_2)

Creating buttons and widgets: https://medium.com/@technologger/how-to-interact-with-jupyter-33a98686f24e

In [ ]:
%whos